In [3]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_hdf('../metadata.h5')
df.head()

,date_added,date_modified,guid,id,meta_info,name,parents,type,url,keyword,img_url,children_id
0,13202479594197706,13242831177794171,00000000-0000-4000-a000-000000000002,1,,書籤列,[1],folder,,[],,"[1884, 1891, 1897, 1903, 1906, 1907, 1908, 190..."
1,13233003241034750,13233003241037911,f0b52077-6793-498a-b8eb-462dfc400922,1884,,2019-nCoV,"[1, 1884]",folder,,"[2019, nCoV]",,"[1885, 1886, 1888, 1890]"
2,13226238181043741,,7f8a21b9-b559-44ab-b711-c41800d188d1,1885,,Coronavirus_Taiwan,"[1, 1884, 1885]",url,https://viator.maps.arcgis.com/apps/opsdashboa...,"[Coronavirus, Taiwan]",,[]
3,13226407607803828,,24a7d7be-2a87-4481-b157-3773db2f4eab,1886,,2019-nCoV | 武漢肺炎地圖,"[1, 1884, 1886]",url,https://kiang.github.io/2019-nCoV/,"[2019, nCoV, 武漢, 地圖, 肺炎]",http://kiang.github.io/2019-nCoV/og_image.png,[]
4,13227360115873492,,a09ae3f0-3a5f-417c-ad10-423d35e155f3,1888,,程序員硬核勸告：現在還不是出門的時候 - 幫趣,"[1, 1884, 1888]",url,https://bangqu.com/a7wXH5.html,"[硬核, 勸告, 現在還, 出門, 時候, 幫趣, 程序]",https://d2ndd3gtcc6iwc.cloudfront.net/liang/20...,[]


In [6]:
df.loc[2, 'url']

'https://viator.maps.arcgis.com/apps/opsdashboard/index.html#/bbd3dfeeec9c494daf178457c7b74c3e'

In [26]:
def get_domain(url):
    m = re.match('https?://(?P<domain>.*?)/', url)
    if m:
        domain = '.'.join(m.groupdict()['domain'].split('.')[-2:])
        return domain
    return None

domain_series = df['url'].apply(get_domain)

In [40]:
tags_unique = domain_series[domain_series.notnull()].unique()
print(tags_unique.shape)
tags_unique[:5]

(257,)


array(['arcgis.com', 'github.io', 'bangqu.com', 'google.com',
       'hubstaff.com'], dtype=object)

In [42]:
bookmark_ids = pd.Series(tags_unique).progress_apply(lambda tag: np.where(domain_series == tag)[0])
print(bookmark_ids.shape)
bookmark_ids.head()

100%|██████████████████████████████████████████████████████████████████████████████| 257/257 [00:00<00:00, 1079.47it/s]


(257,)


0                                                  [2]
1    [3, 13, 92, 153, 159, 161, 269, 290, 356, 385,...
2    [4, 11, 25, 36, 37, 78, 157, 160, 212, 261, 26...
3    [5, 12, 93, 94, 105, 122, 124, 314, 451, 477, ...
4                                                  [7]
dtype: object

In [47]:
tag2ids = pd.Series(bookmark_ids.values, index=tags_unique)
tag2ids.head()

arcgis.com                                                    [2]
github.io       [3, 13, 92, 153, 159, 161, 269, 290, 356, 385,...
bangqu.com      [4, 11, 25, 36, 37, 78, 157, 160, 212, 261, 26...
google.com      [5, 12, 93, 94, 105, 122, 124, 314, 451, 477, ...
hubstaff.com                                                  [7]
dtype: object

In [62]:
f = tag2ids.to_frame()
f = f.reset_index().rename(columns={
    'index': 'name',
    0: 'children'
})
f['children'] = f['children'].apply(lambda arr: arr.tolist())
f['type'] = 'tag'
f.head()

,name,children,type
0,arcgis.com,[2],tag
1,github.io,"[3, 13, 92, 153, 159, 161, 269, 290, 356, 385,...",tag
2,bangqu.com,"[4, 11, 25, 36, 37, 78, 157, 160, 212, 261, 26...",tag
3,google.com,"[5, 12, 93, 94, 105, 122, 124, 314, 451, 477, ...",tag
4,hubstaff.com,[7],tag


In [63]:
f.to_hdf('../static/tag_dfs/web_domain.h5', key='data')